In [ ]:
import cobra
from cobra.flux_analysis import single_gene_deletion_deletion
import pandas as pd

In [ ]:
#import the gapfilled version of the model for testing
model = cobra.io.read_sbml_model("../results/reconstructions/gapfilled_model_name_here.xml")

In [ ]:
#for all exchange reactions, use the boundry reaction function in cobrapy, seems more straightforward
#define the media conditions, first the complete media conditions
# to do this set the exchange reactions open for constituents of the complete King's B media
for met in model.metabolites:
    try  

In [ ]:
#under complete media conditions, run single gene deletions and check for growth
deletion_results_complete_media = single_gene_deletion(model)
#the results from this will be a table with the gene deleted, the flux, and the status of the model(optimum or not)    

In [ ]:
#define the media conditions, now in the Ara media defined by Anderson in the apoplast sampling paper, first Mock apoplasmic fluid
#again, this will be done by opening up exchange reactions for the metabolites that are present in the analysis
for met in model.metabolites:
    try  

In [ ]:
#under mock apoplasmic fluid media conditions, run single gene deletions and check for growth
deletion_results_ara_mock_media = single_gene_deletion(model)
#the results from this will be a table with the gene deleted, the flux, and the status of the model(optimum or not)    

In [ ]:
#define the media conditions, now in the Ara media defined by Anderson in the apoplast sampling paper, now Flg22 apoplasmic fluid
#again, this will be done by opening up exchange reactions for the metabolites that are present in the analysis
for met in model.metabolites:
    try  

In [ ]:
#under flg apoplasmic fluid media conditions, run single gene deletions and check for growth
deletion_results_ara_flg_media = single_gene_deletion(model)
#the results from this will be a table with the gene deleted, the flux, and the status of the model(optimum or not)    